# Limpieza y vectorización de un libro (PLN)

Objetivo: cargar un libro en texto plano (`data/libro.txt`), aplicar **normalización + lematización** (español) y luego **vectorizar** con **BoW** y **TF‑IDF** (con n‑gramas).

Si tu entrega requiere *El Principito*, reemplaza el contenido de `data/libro.txt` por el texto del Principito en UTF‑8 (o usa `data/principito.txt` y ajusta `DATA_PATH`).




In [ ]:
from pathlib import Path
from collections import Counter

import spacy

ROOT = Path().resolve()
DATA_PATH = ROOT / "data" / "libro.txt"
OUTPUT_DIR = ROOT / "outputs"
SPACY_MODEL = "es_core_news_sm"

print("Proyecto:", ROOT)
print("Entrada:", DATA_PATH)
print("Salida:", OUTPUT_DIR)


## 1) Cargar texto

El archivo debe estar en **UTF‑8**.

In [ ]:
text_raw = DATA_PATH.read_text(encoding="utf-8")
text_raw = " ".join(text_raw.split())

print("Chars:", len(text_raw))
print(text_raw[:400])


## 2) Cargar modelo spaCy (español)

Si el modelo no está instalado, se descarga automáticamente.

In [ ]:
try:
    nlp = spacy.load(SPACY_MODEL)
except OSError:
    from spacy.cli import download
    download(SPACY_MODEL)
    nlp = spacy.load(SPACY_MODEL)

doc = nlp(text_raw)
print("Tokens spaCy:", len(doc))
print([t.text for t in doc[:20]])


## 3) Limpieza (normalización + lematización)

Reglas:
- quitar stopwords
- quitar puntuación/espacios/números
- quedarse solo con tokens alfabéticos
- usar `token.lemma_` en minúsculas

In [ ]:
lemmas = []

for token in doc:
    if token.is_space or token.is_punct or token.like_num:
        continue
    if token.is_stop:
        continue
    if not token.is_alpha:
        continue

    lemma = token.lemma_.lower().strip()
    if lemma:
        lemmas.append(lemma)

print("Lemas:", len(lemmas))
print("Primeros 30:", lemmas[:30])


## 4) Guardar outputs

- `outputs/libro_lemmas.txt`
- `outputs/libro_normalizado.txt`
- `outputs/top_30_frecuencias.txt`

## 5) Vectorización (Checkpoint 2 — Pt. 2)

Vectorización por **oraciones lematizadas** usando:
- **Bag of Words (BoW)** con n‑gramas
- **TF‑IDF** con n‑gramas

Y se guardan artefactos:
- `outputs/X_bow.npz`, `outputs/X_tfidf.npz`
- `outputs/vocab_bow.txt`, `outputs/vocab_tfidf.txt`
- `outputs/vectorizacion_meta.json`

In [ ]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

(OUTPUT_DIR / "libro_lemmas.txt").write_text("\n".join(lemmas) + "\n", encoding="utf-8")
(OUTPUT_DIR / "libro_normalizado.txt").write_text(" ".join(lemmas) + "\n", encoding="utf-8")

freq = Counter(lemmas)
top_30 = freq.most_common(30)
(OUTPUT_DIR / "top_30_frecuencias.txt").write_text(
    "\n".join([f"{w}\t{c}" for w, c in top_30]) + "\n",
    encoding="utf-8",
)

print("Únicos:", len(freq))
print("Top 10:", top_30[:10])


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA

import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
import json

# Este checkpoint 2 (pt. 2) vectoriza por ORACIONES lematizadas


In [ ]:
corpus_lematizado = []

sents = list(doc.sents)
if not sents:
    # Fallback (raro): si no hay segmentación de oraciones, intentamos con sentencizer
    if "sentencizer" not in nlp.pipe_names:
        nlp.add_pipe("sentencizer")
    doc = nlp(text_raw)
    sents = list(doc.sents)

for oracion in sents:
    lemas_oracion = []
    for token in oracion:
        if token.is_space or token.is_punct or token.like_num:
            continue
        if token.is_stop:
            continue
        if not token.is_alpha:
            continue
        lemma = token.lemma_.lower().strip()
        if lemma:
            lemas_oracion.append(lemma)

    if lemas_oracion:
        corpus_lematizado.append(" ".join(lemas_oracion))

print(f"Total de oraciones procesadas: {len(corpus_lematizado)}")
print("Ejemplo:", corpus_lematizado[0][:160] + ("..." if len(corpus_lematizado[0]) > 160 else ""))

In [ ]:
# Bag of Words (conteos) con n-gramas
bow_vectorizer = CountVectorizer(ngram_range=(1, 2))
X_bow = bow_vectorizer.fit_transform(corpus_lematizado)

print("X_bow:", X_bow.shape)
print("Vocab BoW:", len(bow_vectorizer.get_feature_names_out()))

In [ ]:
# TF-IDF (importancia) con n-gramas
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = tfidf_vectorizer.fit_transform(corpus_lematizado)

print("X_tfidf:", X_tfidf.shape)
print("Vocab TF-IDF:", len(tfidf_vectorizer.get_feature_names_out()))

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 (necesario para 3D)


def _top_k_by_sum(X, k: int):
    scores = np.asarray(X.sum(axis=0)).ravel()
    k = min(k, scores.size)
    idx = np.argsort(-scores)[:k]
    return idx


def _top_k_by_mean(X, k: int):
    scores = np.asarray(X.mean(axis=0)).ravel()
    k = min(k, scores.size)
    idx = np.argsort(-scores)[:k]
    return idx


def _pca_3d_coords(X_words):
    pca = PCA(n_components=3, random_state=0)
    return pca.fit_transform(X_words)


def graficar_palabras_3d(ax, X_oraciones_x_terminos, vocabulario, titulo, color_puntos, idx_terminos):
    X_sel = X_oraciones_x_terminos[:, idx_terminos]
    vocab_sel = vocabulario[idx_terminos]

    # (palabras x oraciones)
    X_words = X_sel.T.toarray()
    coords = _pca_3d_coords(X_words)

    x, y, z = coords[:, 0], coords[:, 1], coords[:, 2]
    ax.scatter(x, y, z, c=color_puntos, s=70, edgecolors="k", alpha=0.85, depthshade=True)

    for i, palabra in enumerate(vocab_sel):
        ax.text(x[i], y[i], z[i] + 0.05, palabra, fontsize=8)

    ax.set_title(titulo)
    ax.set_xlabel("Comp. Principal 1")
    ax.set_ylabel("Comp. Principal 2")
    ax.set_zlabel("Comp. Principal 3")


top_k = 40

fig = plt.figure(figsize=(18, 8))

ax1 = fig.add_subplot(121, projection="3d")
vocab_bow = bow_vectorizer.get_feature_names_out()
idx_bow = _top_k_by_sum(X_bow, top_k)
graficar_palabras_3d(ax1, X_bow, vocab_bow, "Espacio BoW 3D (Top términos)", "orange", idx_bow)

ax2 = fig.add_subplot(122, projection="3d")
vocab_tfidf = tfidf_vectorizer.get_feature_names_out()
idx_tfidf = _top_k_by_mean(X_tfidf, top_k)
graficar_palabras_3d(ax2, X_tfidf, vocab_tfidf, "Espacio TF-IDF 3D (Top términos)", "teal", idx_tfidf)

plt.tight_layout()
plt.show()

In [ ]:
# Guardar artefactos de vectorización (sparse matrices + vocabularios)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

sparse.save_npz(OUTPUT_DIR / "X_bow.npz", X_bow)
sparse.save_npz(OUTPUT_DIR / "X_tfidf.npz", X_tfidf)

(OUTPUT_DIR / "vocab_bow.txt").write_text(
    "\n".join(bow_vectorizer.get_feature_names_out()) + "\n",
    encoding="utf-8",
)
(OUTPUT_DIR / "vocab_tfidf.txt").write_text(
    "\n".join(tfidf_vectorizer.get_feature_names_out()) + "\n",
    encoding="utf-8",
)

meta = {
    "n_oraciones": len(corpus_lematizado),
    "bow_shape": list(X_bow.shape),
    "tfidf_shape": list(X_tfidf.shape),
    "ngram_range": [1, 2],
}
(OUTPUT_DIR / "vectorizacion_meta.json").write_text(
    json.dumps(meta, ensure_ascii=False, indent=2) + "\n",
    encoding="utf-8",
)

print("Guardado en:", OUTPUT_DIR)